# Preparacion de directorio

In [ ]:
def setup_project(main_fold, project_fold, mount_drive=True):

    """
    Mounts Google Drive, sets up sys.path, and changes directory to the project folder.
    Only prints the current working directory and the project folder name.
    """
    import os
    import sys
    from google.colab import drive

    if mount_drive:
        drive.mount('/content/drive')

    base_dir = '/content/drive/MyDrive'
    parent_project_dir = os.path.join(base_dir, main_fold)
    project_dir = os.path.join(parent_project_dir, project_fold)

    # Add parent directory to sys.path
    if parent_project_dir not in sys.path:
        sys.path.append(parent_project_dir)

    # Change working directory if exists
    if os.path.exists(project_dir):
        os.chdir(project_dir)
    else:
        print(f"Project directory not found: {project_dir}")
        return

    # Add project directory to sys.path
    if project_dir not in sys.path:
        sys.path.append(project_dir)

    # Print only the working directory and project folder name
    print(f"Current working directory: {os.getcwd()}")
    print(f"Project folder: {project_fold}")


In [ ]:
main_fold = 'Machine Learning Projects'
project_fold = 'Analisis Cluster Data Importacion'

setup_project(main_fold, project_fold)

# Dependencias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import cupy as cp
from cuml.manifold import UMAP as cuUMAP
from cuml.cluster import HDBSCAN as cuHDBSCAN
from bertopic.representation import OpenAI
from bertopic import BERTopic
import openai

import re
from lab_utils import get_data, tariff_description, sugerencias_builder

CARGA DE DATA

In [ ]:
ruta1 = './datos/data_importacion_sin_clasificar.csv'
ruta2 = './datos/dicaranceles.xlsx'

df= get_data(ruta1)
df_diccionario = get_data(ruta2)

shape1 = df.shape
print(f'El dataframe tiene {shape1[0]} filas y {shape1[1]} columnas')


MUESTREO Y AJUSTE DE DATA

In [ ]:
df = df.sample(n=10000, random_state=42)
df.reset_index

columna_codigo_arancelario = 'posicion_arancelaria'
columna_descripcion = 'descripcion_x'
df = tariff_description(df, df_diccionario, columna_codigo_arancelario)
df = sugerencias_builder(df,columna_descripcion)

TRANSFORMACION Y TRATAMIENTO DE DATOS

In [ ]:
def corpus_cleaner(
    df,
    column_corpus: str = "corpus",
    palabras_a_eliminar: list = None,
    replacement_dict: dict = None,
    new_column: str = "corpus_limpio"
):
    """
    Cleans a text column by removing unwanted words/stopwords and applying replacements.

    Parameters:
        df: DataFrame with the text column to clean.
        column_corpus: Name of the column to clean.
        palabras_a_eliminar: List of words to remove. If None, uses default list.
        replacement_dict: Dictionary of replacements, e.g. {'telãfono': 'telefono'}.
        new_column: Name of the column to store the cleaned output.

    Returns:
        DataFrame with the new cleaned column added.
    """
    import re
    import pandas as pd

    if palabras_a_eliminar is None:
        palabras_a_eliminar = [
            # Artículos
            'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas', 'lo', 'al', 'del',
            # Preposiciones y conjunciones
            'de', 'a', 'en', 'y', 'que', 'con', 'por', 'para', 'sin', 'sobre', 'bajo', 'entre', 'hacia', 'hasta', 'desde', 'durante',
            # Pronombres y posesivos
            'se', 'su', 'sus', 'le', 'les', 'me', 'te', 'nos', 'os', 'mi', 'tu', 'nuestro', 'vuestro',
            # Adverbios y conectores redundantes
            'muy', 'mas', 'menos', 'también', 'además', 'incluso', 'inclusive', 'siempre', 'nunca', 'ahora', 'después', 'antes', 'aquí', 'allí', 'pniña',
            # Palabras genéricas
            'articulos', 'similares', 'partes', 'punto', 'tipo', 'vez', 'cosas', 'hecho', 'caso', 'ejemplo', 'manera', 'forma', 'fin', 'modo', 'demas', 'similar', 'otras',
            'excepto',	'partida', 'incluidas', 'incluidos', 'patrones', 'esta', 'clase', 'artic', 'partidas', 'aunque', 'esten', 'en', 'parte', 'mm', 'set', 'transporte',
            # Verbos comunes
            'ser', 'estar', 'tener', 'haber', 'hacer', 'poder', 'decir', 'ver', 'saber'
        ]
    if replacement_dict is None:
        replacement_dict = {
            'telãfono': 'telefono'
            # Add more replacements here as needed
        }

    # Step 1: Remove punctuation
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = re.sub(r'[^\w\s]', '', text)
        # Build regex for exact words, case-insensitive
        regex_palabras = r'(?<!\w)(' + '|'.join(map(re.escape, palabras_a_eliminar)) + r')(?!\w)'
        text = re.sub(regex_palabras, '', text, flags=re.IGNORECASE)
        text = re.sub(r'\s+', ' ', text).strip()
        # Remove numbers
       # text = re.sub(r'\d+', '', text)
        return text

    result = df.copy()
    result[new_column] = result[column_corpus].apply(clean_text)

    # Step 2: Replacements
    for old_word, new_word in replacement_dict.items():
        result[new_column] = result[new_column].str.replace(old_word, new_word, regex=False)

    return result

irrelevantes = set([
    "de", "las", "los", "y", "en", "a", "o", "del", "la", "el", "un", "una", "con", "por", "para", "al"
])

keywords_simples = {
    "vehiculos", "vehículo", "vehículos", "automoviles", "automóvil", "automóviles",
    "carro", "carros", "auto", "autos", "camioneta", "camionetas", "camión", "camiones"
}

keywords_compuestas = [
    "repuesto para vehiculos", "repuesto para vehículo", "repuesto para carro", "repuesto para auto",
    "partes de carro", "partes de vehiculos", "partes de vehículo", "partes de auto", "partes de camioneta",
    "partes de camión", "accesorios de vehículos", "accesorios de auto", "accesorios de carro",
    "accesorios de camioneta", "accesorios de camión", "componentes automotrices", "industria automotriz"
]

def get_first_two_words(text):
    palabras = text.split()
    return " ".join(palabras[:2])

def get_first_two_relevant(text):
    palabras = re.split(r'\W+', text.lower())
    relevantes = [p for p in palabras if p and p not in irrelevantes]
    return " ".join(relevantes[:2])

def keywords_en_corpus(corpus):
    encontrados = set()
    corpus_lower = corpus.lower()
    for frase in keywords_compuestas:
        if frase in corpus_lower:
            encontrados.add(frase)
    palabras = set(corpus_lower.split())
    for kw in keywords_simples:
        if kw in palabras:
            encontrados.add(kw)
    return " ".join(encontrados)

def patron_manufacturas(text2):
    # Busca "LAS DEMAS MANUFACTURAS DE [ARTÍCULO]"
    match = re.search(r"LAS DEMAS MANUFACTURAS DE ([\w\s]+)", text2, re.IGNORECASE)
    if match:
        articulo = match.group(1).strip()
        # Elimina stopwords y deja solo lo relevante
        palabras = [w for w in articulo.split() if w.lower() not in irrelevantes]
        if palabras:
            return f"manufacturas de {' '.join(palabras)}"
    return ""

def patron_placas_laminas(text2):
    # Busca "PLACAS, LAMINAS, HOJAS Y TIRAS"
    if re.search(r"PLACAS[, ]+LAMINAS", text2, re.IGNORECASE):
        return "placas laminas"
    return ""

def patron_manufacturas(text2):
    # Busca "LAS DEMAS MANUFACTURAS DE [ARTÍCULO]"
    match = re.search(r"LAS DEMAS MANUFACTURAS DE ([^,Y\n]+)", text2, re.IGNORECASE)
    if match:
        articulo = match.group(1).strip()
        # Elimina stopwords y deja solo lo relevante
        palabras = [w for w in articulo.split() if w.lower() not in irrelevantes]
        if palabras:
            return f"manufacturas de {' '.join(palabras)}"
    return ""

def extract_texto_lectura(row):
    text2 = str(row["text2"])
    parte1 = get_first_two_words(str(row["corpus_limpio"]))
    parte2 = patron_manufacturas(text2)
    parte3 = keywords_en_corpus(str(row["corpus_limpio"]))
    resultado = parte1
    if parte2:
        resultado = f"{resultado} {parte2}"
    if parte3:
        tokens_resultado = set(resultado.lower().split())
        extra = " ".join([x for x in parte3.split() if x not in tokens_resultado])
        if extra:
            resultado = f"{resultado} {extra}"
    return resultado.strip().lower()

In [ ]:
df = corpus_cleaner(df)

# Aplica la función al DataFrame
df["texto_lectura"] = df.apply(extract_texto_lectura, axis=1)

abstracts = df['texto_lectura'].tolist()

PREPARACION METOLOGIA BERTOPIC

In [ ]:
# --- CELDA 3: PREPARAR NLTK Y STOPWORDS ---
nltk.download('punkt')
nltk.download('stopwords')

spanish_stopwords = set(stopwords.words('spanish'))

# Si tienes un set de stopwords extra, define o importa aquí:
# Ejemplo:
# EXTRA_STOPWORDS = {"producto", "venta", ...}
try:
    combined_stopwords = spanish_stopwords.union(EXTRA_STOPWORDS)
except NameError:
    print("EXTRA_STOPWORDS no definido, usando solo stopwords estándar.")
    combined_stopwords = spanish_stopwords


In [ ]:
# --- CELDA 7: VECTORIZADOR ---
vectorizer_model = CountVectorizer(
    stop_words=list(combined_stopwords),
    min_df=2,
    max_df=0.95,
    ngram_range=(1, 2)
)

In [ ]:
# --- CELDA 8: MODELO DE REPRESENTACIÓN OPENAI ---
openai_api_key = "sk-proj-..."

prompt = """
Tengo un conjunto de documentos, cada uno relacionado con productos:
[DOCUMENTOS]
Estos productos están descritos por las siguientes palabras clave: [KEYWORDS]

Tu tarea es extraer un nombre de categoría corto y altamente descriptivo (máximo 5 palabras) que represente al grupo de productos.
No quiero un “tópico”, ni un resumen, sino una palabra o frase representativa que funcione como categoría, clase o etiqueta para el grupo de artículos.
Todos los documentos son productos, así que piensa en términos de categorías comerciales.

resultado, solo el nombre de la categoria. No incluyas nada más

"""

client = openai.OpenAI(api_key=openai_api_key)
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)
representation_model = {"OpenAI": openai_model}


In [ ]:


# Inicializar modelo
#print("🔄 Cargando modelo Qwen3...")
#embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

openai.api_key = openai_api_key


print("✅ Modelo cargado exitosamente")

# =====================================================
# CONFIGURACIÓN DE PROMPT
# =====================================================
custom_prompt = (
" The following are items descriptions in spanish so create 220 categories base on the function of the item :  ")
prompts_alternatives = [
    "el siguiente texto son descripciones de productos, dame su categoria: ",
    "Clasifica este producto importado según su categoría comercial: ",
    "El siguiente es un producto de importación, determina su categoría: ",
    "Categoría del producto: ",
    "Identifica la categoría comercial de este artículo: ",
    "¿A qué categoría pertenece este producto según su uso?: ",
    "",  # Sin prompt
    "Clasifica según categorías como electrónicos, textiles, alimentos, etc.: ",
    "Basándose en la descripción, la categoría de este producto es: "
]

print(f"📝 Prompt seleccionado: '{custom_prompt}'")

# Supón que abstracts es tu lista de descripciones
texts = [custom_prompt + t for t in abstracts]

# =====================================================
# OBTENER EMBEDDINGS CON OPENAI
# =====================================================
def get_openai_embeddings(texts, model="text-embedding-3-large"):
    embeddings = []
    batch_size = 100  # OpenAI permite hasta 2048 tokens por texto y hasta 2048 textos por request, pero lo conservador es 100
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = openai.embeddings.create(
            input=batch,
            model=model,
        )
        batch_embeddings = [d.embedding for d in response.data]
        embeddings.extend(batch_embeddings)
    return embeddings

embeddings = get_openai_embeddings(texts)

print("✅ Embeddings generados con OpenAI")


In [ ]:
# Normalizar embeddings
embeddings = np.array(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
# UMAP
umap_model = cuUMAP(
    n_neighbors=50,    # mayor es el numero mas macro es la clasificacion
    n_components=500,  # a partir de 60 el modelo si hace buena generalizacion
    min_dist=0.01,     # densidad de los clusters
    metric='euclidean',
    random_state=42,
    verbose=True
)
reduced_embeddings = umap_model.fit_transform(embeddings)

# HDBSCAN
hdbscan_model = cuHDBSCAN(
    min_cluster_size=12,
    min_samples=7,
    cluster_selection_epsilon=0.15,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

GENERALIZAR CLASES

In [ ]:
# --- CELDA 9: CREAR Y AJUSTAR BERTopic ---
topic_model = BERTopic(
    embedding_model=None,
    umap_model=None,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    top_n_words=10,
    verbose=True,
    language="spanish"
)

topics, probs = topic_model.fit_transform(abstracts, reduced_embeddings)

In [ ]:
chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
chatgpt_topic_labels[-1] = "Outlier Topic"
topic_model.set_topic_labels(chatgpt_topic_labels)

EXPLORACION DE GRUPOS (SUBCATEGORIAS)

In [ ]:
topic_model.get_topic(11, full=True)

In [ ]:
topic_5_docs_indices = [i for i, topic in enumerate(topics) if topic == 11]
print(f"Abstracts for Topic 5 ({len(topic_5_docs_indices)} documents):")
for i in topic_5_docs_indices:
  print(f"- {abstracts[i]}")

In [ ]:
topic_model.visualize_topics(custom_labels=True)

CREACION DE CATEGORIAS

In [ ]:
import pandas as pd
import openai
import ast
import re

def enrich_df_with_subcategory(df, topics, topic_model):
    """
    Añade una columna de subcategoría (nombre de tópico) al DataFrame y devuelve el mapping topic_id_to_customname.
    """
    # topic_model.get_topic_info() devuelve un DataFrame con Topic y Name
    topic_info = topic_model.get_topic_info()
    # Extract the custom names generated by OpenAI, defaulting to the BERTopic name if OpenAI is not available
    if "OpenAI" in topic_model.topic_aspects_:
         topic_id_to_customname = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
    else:
         topic_id_to_customname = dict(zip(topic_info.Topic, topic_info.Name))

    # Ensure -1 is handled for outliers
    topic_id_to_customname[-1] = "Outlier Topic"


    df = df.copy()
    df["Topic"] = topics
    # Use .get() with a default value for safety
    df["subcategory"] = df["Topic"].map(topic_id_to_customname).fillna("Sin subcategoría")
    return df, topic_id_to_customname

def create_topic_to_category_mapping(topic_id_to_customname, openai_api_key, df, n_categories=None, prompt_extra=""):
    """
    Usa OpenAI para agrupar subcategorías (por Topic) en categorías principales de sentido común.
    Si n_categories es None, el modelo decide el número de categorías.
    """
    topics_in_data = set(df['Topic'].unique())
    subcats = [f"{k}: {v}" for k, v in topic_id_to_customname.items() if k != -1]

    if n_categories is not None:
        prompt = (
            f"Quiero que agrupes las siguientes subcategorías de productos en exactamente {n_categories} categorías principales. Unas categorias que siempre deben existir es Electrodomesticos, Cocina, Muebles y Decoracion, Automotriz, Ferreteria, Deportes, Papeleria, Jugetes y Juegos."
            "Dame el resultado como un diccionario Python Topic: (category_code, category_name), donde category_code es un int y category_name un string. "
            "Incluye absolutamente todos los Topic que te paso, incluyendo -1. "
            "Si falta alguno, asígnale (9999, 'Sin Categoria'). "
            "El resultado debe estar SOLO contenido en un bloque de código Python (triple backtick y python), sin ninguna explicación, solo el diccionario.\n"
            f"Subcategorías:\n" + "\n".join(subcats) + f"\n{prompt_extra}"
        )
    else:
        prompt = (
            "Analiza la siguiente lista de subcategorías de productos. Agrúpalas en las categorías principales más naturales perosiempre deben existir las categorias Electrodomesticos, Cocina, Muebles y Decoracion, Automotriz, Ferreteria, Deportes, Papeleria, Jugetes y Juegos. estas son solo un baseline, lo demas hazlo a tu criterio"
            "según el significado y similitud de sus nombres. El número de categorías lo decides tú, según lo que tenga más sentido. "
            "Por ejemplo, todo lo que sea ropa en una sola categoría, igual con electrónica, juguetes, etc. "
            "Devuélveme solo un diccionario Python Topic: (category_code, category_name), donde category_code es un int único y category_name un string. "
            "Incluye absolutamente todos los Topic que te paso, incluyendo -1. Si falta alguno, asígnale (9999, 'Sin Categoria'). "
            "El resultado debe estar SOLO contenido en un bloque de código Python (triple backtick y python), sin ninguna explicación, solo el diccionario.\n"
            f"Subcategorías:\n" + "\n".join(subcats) + f"\n{prompt_extra}"
        )

    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=5000,
    )
    content = response.choices[0].message.content

    code_match = re.search(r"```python\s*\n([\s\S]+?)```", content)
    if code_match:
        mapping_str = code_match.group(1)
    else:
        code_match = re.search(r"(\{[\s\S]+\})", content)
        if code_match:
            mapping_str = code_match.group(1)
        else:
            raise ValueError(f"No se pudo extraer el diccionario de la respuesta de OpenAI. Respuesta fue:\n{content}")

    try:
        topic_to_category = ast.literal_eval(mapping_str)
    except Exception as e:
        raise ValueError(f"No se pudo interpretar el diccionario extraído. String extraído:\n{mapping_str}\nError: {str(e)}")

    for code in topics_in_data:
        if code not in topic_to_category:
            if code == -1:
                topic_to_category[code] = (9999, "Outlier Category")
            else:
                topic_to_category[code] = (9999, "Sin Categoria")
    return topic_to_category

def enrich_df_with_category(df, topic_to_category):
    """
    Añade columnas de código de categoría y nombre de categoría al DataFrame usando el mapping topic_to_category.
    """
    df = df.copy()
    # Use .get() with a default value for safety
    df["category_code"] = df["Topic"].map(lambda x: topic_to_category.get(x, (9999, "Sin Categoria"))[0])
    df["category"] = df["Topic"].map(lambda x: topic_to_category.get(x, (9999, "Sin Categoria"))[1])
    return df

In [ ]:
# --- PHASE 1: ORIGINAL DATA ---

df, topic_id_to_customname = enrich_df_with_subcategory(df, topics, topic_model)

# Ahora el modelo decide las categorías según sentido común (no pongas n_categories)
topic_to_category = create_topic_to_category_mapping(
    topic_id_to_customname,
    openai_api_key,
    df,
    n_categories=None  # <--- Esto hace que se use el prompt de sentido común
)

df = enrich_df_with_category(df, topic_to_category)


GUARDAR DATOS ETIQUETAS CON CATEGORIA Y SUBCATEGORIA

In [ ]:
df.to_csv('tablas/sample_OPENAI_2_SHOT.csv', index=False)

# ANEXO : INSTALACION DE DEPENDENCIAS Y RAPIDS //

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
!pip install bertopic
!pip install umap-learn
!pip install hdbscan